<a href="https://colab.research.google.com/github/cicattzo/nlp_project/blob/main/6_864_project_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install sumy

  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=d23d55a8d3f9034dd6fb96a0e168b40dcd688b98a50116cba55a1eb2ba2ecf3b
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21680 sha256=967238cbd9b7ca61003e135f50d58e72087bc126fcf4669aff3961b6a3803050
  Stored in directory: /root/.cache/pip/wheels/5a/4d/a1/510b12c5e65e0b2b3ce539b2af66da0fc57571e528924f4a52
Successfully built pycountry breadability


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
import pandas as pd
import numpy as np
import re
# import xml.etree.ElementTree as ET
# from talon.signature.bruteforce import extract_signature
# import email

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Basic structure for summarization with Sumy

In [ ]:
LANGUAGE = "english"
SENTENCES_COUNT = 1
tokenizer = Tokenizer(LANGUAGE)
stemmer = Stemmer(LANGUAGE)
# change this line to any other summarizer
summarizer = TextRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

loading pre-processed BC3 emails corpus

In [ ]:
# bc3_df = pd.read_csv("/content/gdrive/My Drive/6864_project/bc3_processed.csv")
bc3_df = pd.read_csv("bc3_test.csv")

In [ ]:
txt = bc3_df.iloc[0]['body']
summary = bc3_df.iloc[0]['summary']

In [ ]:
parser = PlaintextParser.from_string(txt, tokenizer)

In [ ]:
print("The Text:\n")
print(txt)
print("\nThe Summary:\n")
print(summary)
print("\nText Rank\n")

for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)
    break

The Text:

The IETF meetings tend to become too large, creating logistics and planning problems. I suggest that future meetings are held for two weeks, with applications and user services issues the first week, and all other issues the second week. Those who so wish could attend both weeks, and other people could attend only one week. Those who choose to attend both weeks would be able to cover more groups and do better liaisons between the different areas. The Friday of the first week could discuss applications issues which might be of special interest to the other areas, and the Monday of the second week would schedule other groups which might be of special interest to applications people, so some people could attend Monday-Monday or Friday-Friday. 
------------------------------------------------------------------------ 
Jacob Palme &lt;jpalme@dsv.su.se&gt; (Stockholm University and KTH) 
for more info see URL: http://www.dsv.su.se/~jpalme

The Summary:

Jacob suggested to hold two 

In [ ]:
bc3_df.iloc[0]['subject']

'Extending IETF meetings to two weeks?'

In [ ]:
bc3_df['listno'] + "-" + bc3_df['email_num'].astype(str)

0       007-7484738-1
1       007-7484738-1
2       007-7484738-1
3       007-7484738-2
4       007-7484738-2
            ...      
664    107-16164699-6
665    107-16164699-6
666    107-16164699-6
667    107-16164699-7
668    107-16164699-7
Length: 669, dtype: object

In [ ]:
%%capture
!pip install torch==1.4.1
!pip install datasets==1.0.2
!pip install transformers==4.5.1

In [ ]:
import datasets

train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train[:2%]")
train_data = [x for x in train_data]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602. Subsequent calls will reuse this data.


In [ ]:
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
val_data = [x for x in val_data]

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)


## Evaluating BLEU and ROUGE scores

In [ ]:
%%bash
pip install sacrebleu
pip install rouge_score

In [ ]:
# import datasets
import sacrebleu
from rouge_score import rouge_scorer

In [ ]:
sacrebleu.raw_corpus_bleu([pred], [[targ]], .01).score

0.08885965541568026

In [ ]:
rscorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

In [ ]:
rscorer.score(targ, pred)

{'rouge1': Score(precision=0.3076923076923077, recall=0.13333333333333333, fmeasure=0.18604651162790697),
 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0)}

## Converting BC3 corpus to a dict

In [ ]:
# keeping the subject and body separate, but they can be merged
bc3_df['unique_key'] = bc3_df['listno'] + "-" + bc3_df['email_num'].astype(str)
bc3_df['article'] = bc3_df['subject'] + " \n" + bc3_df['body']
# bc3_dict = bc3_df.groupby('unique_key').agg({'subject':lambda x: x.iloc[0], 'body':lambda x: x.iloc[0], 'summary':lambda x: x.to_list()}).to_dict('records')
# bc3_dict = bc3_df.agg({'body':lambda x: x.iloc[0], 'summary':lambda x: x.iloc[0]}).to_dict('records')
bc3_dict = bc3_df[['body', 'summary']].to_dict('records')

In [ ]:
bc3_dict[0]

{'body': '> In fact, I have it on the shelf behind me at work as I type this, beside my XHTML, XSLT, CSS, usability, and disability references. \nIs this bookshelf documented on Kynn.COM, or would you consider sharing with us a short list of the references you find the most helpful? \nAl',
 'summary': "Al asks if Kynn's book collection is documented on Kynn's site."}

The structure of the dict is: \
{ \
  'subject': Dict(str: unique_key => str: subject_text) \
  'body':    Dict(str: unique_key => str: body_text) \
  'summary': Dict(str: unique_key => list: candidate_summaries ) \
  } \\

  Look at the example below for more clarity
  

In [ ]:
keys = list(bc3_dict['summary'].keys())
# candidate summaries for the first email
bc3_dict['summary'][keys[0]]

['Jacob suggested to hold two week meetings, the first will discuss user service issues and all other issues the following week, he said people can attend one week or both.',
 'Jacob suggests that future IETF meetings be spread over two weeks.',
 'The topic is the logistics of scheduling IETF meetings A suggestion is made to hold future meetings for two weeks to spread out the volume of participants']

## Calculate metrics over the corpus for different baseline summarization models

In [ ]:
LANGUAGE = "english"
SENTENCES_COUNT = 2
tokenizer = Tokenizer(LANGUAGE)
stemmer = Stemmer(LANGUAGE)

rscorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

In [ ]:
def evaluate_baseline(summarizer, corpus, full_text_key, label_key, return_preds=False):
  rouge_score_list = []
  bleu_score_list = []
  predictions = []

  for email in corpus:
    txt = email[full_text_key]
    summary = email[label_key]
    parser = PlaintextParser.from_string(txt, tokenizer)
    pred = summarizer(parser.document, SENTENCES_COUNT)
    pred = str(pred[0]) if len(pred) > 0 else ' '
    predictions.append(pred)
    rouge_score_list.append(rscorer.score(summary, pred))
    bleu_score_list.append(sacrebleu.raw_corpus_bleu([pred], [[summary]], .01).score)

  ret_dict = {
      'rouge2-fscore': np.mean([sc['rouge2'].fmeasure for sc in rouge_score_list]),
      'rouge1-fscore': np.mean([sc['rouge1'].fmeasure for sc in rouge_score_list]),
      'bleu-score': np.mean(bleu_score_list)
  }
  if return_preds:
    ret_dict['predictions'] = predictions

  return ret_dict

In [ ]:
textrank = TextRankSummarizer(stemmer)
textrank.stop_words = get_stop_words(LANGUAGE)

print(evaluate_baseline(textrank, bc3_dict, 'body', 'summary'))

{'rouge2-fscore': 0.07208236663388459, 'rouge1-fscore': 0.20086196801293388, 'bleu-score': 1.5319080528905527}


In [ ]:
TR_preds = evaluate_baseline(textrank, bc3_dict, 'body', 'summary', return_preds=True)['predictions']
bc3_df['textrank_prediction'] = TR_preds

In [ ]:
lsa = LsaSummarizer(stemmer)
lsa.stop_words = get_stop_words(LANGUAGE)

print(evaluate_baseline(lsa, bc3_dict, 'body', 'summary'))

/usr/local/lib/python3.7/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (2) is lower than number of sentences (4). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


{'rouge2-fscore': 0.07384572079143653, 'rouge1-fscore': 0.2096656890345382, 'bleu-score': 2.14322056781507}


In [ ]:
lsa_preds = evaluate_baseline(lsa, bc3_dict, 'body', 'summary', return_preds=True)['predictions']
bc3_df['lsa_prediction'] = lsa_preds

/usr/local/lib/python3.7/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (2) is lower than number of sentences (4). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


In [ ]:
lexrank = LexRankSummarizer(stemmer)
lexrank.stop_words = get_stop_words(LANGUAGE)

print(evaluate_baseline(lexrank, bc3_dict, 'body', 'summary'))

{'rouge2-fscore': 0.07281933492683379, 'rouge1-fscore': 0.20197695757816633, 'bleu-score': 1.8114631458768484}


In [ ]:
LR_preds = evaluate_baseline(lexrank, bc3_dict, 'body', 'summary', return_preds=True)['predictions']
bc3_df['lexrank_prediction'] = LR_preds

In [ ]:
bc3_df.to_csv("bc3_test_baseline_predictions.csv", index=False)

**Evaluating on CNN/Daily Mail**

In [ ]:
textrank = TextRankSummarizer(stemmer)
textrank.stop_words = get_stop_words(LANGUAGE)

print(evaluate_baseline(textrank, val_data, 'article', 'highlights'))

{'rouge2-fscore': 0.08950552559991548, 'rouge1-fscore': 0.28090557114569775, 'bleu-score': 2.8498503134101827}


In [ ]:
lsa = LsaSummarizer(stemmer)
lsa.stop_words = get_stop_words(LANGUAGE)

print(evaluate_baseline(lsa, val_data, 'article', 'highlights'))

{'rouge2-fscore': 0.06472354195642918, 'rouge1-fscore': 0.2403836157898129, 'bleu-score': 2.0917125996980186}


In [ ]:
lexrank = LexRankSummarizer(stemmer)
lexrank.stop_words = get_stop_words(LANGUAGE)

print(evaluate_baseline(lexrank, val_data, 'article', 'highlights'))

{'rouge2-fscore': 0.07808006130581277, 'rouge1-fscore': 0.2448683788580817, 'bleu-score': 2.2365112914004275}


In [ ]:
""